### Load Dependencies

In [1]:
import swisseph as swe
import pandas as pd
import numpy as np

import networkx as nx
import matplotlib.pyplot as plt
from scipy.signal import argrelextrema

from calc import find_new_moon_conjunctions

# configure ephemeris
swe.set_ephe_path('./ephe')

# https://ssd.jpl.nasa.gov/planets/eph_export.html
# Created April 2013; includes librations and 1980 nutation.
# Referred to the International Celestial Reference Frame  version 2.0.
# Covers JED -3100015.5, (-13200 AUG 15) to JED 8000016.5,(17191 MAR 15).
swe.set_jpl_file('DE431.eph')

weekname = ["mon", "tue", "wed", "thu", "fri", "sat", "sun"]

### Configure Script

In [93]:
YEAR_START = -11084
YEAR_START -= 12    # align 294 year cycle with our era
YEAR_END = 10000
# YEAR_START = 1000
# YEAR_START = 2024
# MY_GEO_LOC = [-122.258301, 37.801239, 13]  # Oakland, CA

### Calculate Solar Equinox / Lunar Phase Dataset

In [ ]:
# INITIALIZE VARIABLES
nm = []
first_eq = None
prev_eq = None
moon_year = 0
moon_day = 0
moon_total_days = 0
total_lunar_year_days = 0

# Get Equinox / New Moons for year prior tor YEAR_START
eq = swe.solcross_ut(x2cross=0,tjdut=swe.julday(YEAR_START-1, 1, 1))
next_eq = swe.solcross_ut(x2cross=0,tjdut=swe.julday(YEAR_START, 1, 1))
last_nm = find_new_moon_conjunctions(swe, eq, next_eq)[-1]
for jyear in range(YEAR_START, YEAR_END):
    jd = swe.julday(jyear, 1, 1)
    
    # get vernal equinox for this year
    eq = swe.solcross_ut(
        x2cross=0,
        tjdut=jd
    )
    
    # get vernal equinox for next year
    next_eq = swe.solcross_ut(
        x2cross=0,
        tjdut=swe.julday(jyear + 1, 1, 1)
    )
        
    # save equinox for YEAR_START to maintain running day count
    first_eq = eq if first_eq is None else first_eq

    # get all new moons in the Lunar year ( all new moons between this and the next vernal equinox )
    # you will get back either 12 or 13 months. 
    nm_conj = find_new_moon_conjunctions(swe, eq, next_eq)
        
    # Calculate the number of days in this lunar year by taking the difference
    # between the last lunar month's new moon date from the first lunar months new moon date.
    lunar_year_num_days = nm_conj[-1] - nm_conj[0]       
    
    eq_str = swe.revjul(eq, swe.GREG_CAL)
    moon_year += 1  
    for idx, m in enumerate(nm_conj):        
        last_day_in_month = m - last_nm
        
        nm.append({
            "ve": eq,     
            "ve_date": f"{eq_str[1]}/{eq_str[2]}/{eq_str[0]}",
            "jd": m,
            "nm_eq": m - eq,
            "sun_year": jyear,
            "lunar_year": moon_year,            
            "lunar_month": idx + 1,
            "lunar_days": last_day_in_month,
            "total_days": moon_total_days + last_day_in_month,
            "day_of_week": swe.day_of_week(eq)            
        })
    moon_total_days += (nm_conj[-1] - last_nm)
    last_nm = nm_conj[-1]

nm_df = pd.DataFrame(nm)

### Find where both the sun and moon cycles reset that matches the 294 year cycle proposed by Enoch
This is important in order to know what year in the 294 year cycle we are in.

First we're going to overlay the dataset with a running 364 day year count and the 294 enoch year count.

In [ ]:
# Calculate Enoch 364 Day 
nm_df["eday"] = np.floor(nm_df["total_days"] % 364) + 1 

# Calculate Enoch 294 Year
nm_df["eyear"] = (np.round((nm_df["total_days"] / 364)) % 294) + 1
nm_df[:13]

### Next we want to examime the lunar cycle. 
Lets see if we can find a pattern looking at the distance of the first month in the lunar year 
and the start of the vernal equinox. Here we're going to calculate distance between the number of days between
the date of vernal equinox and the date of the first new moon of the lunar year

In [ ]:
nm_df[nm_df.lunar_month == 1][:150].plot("sun_year", y=["nm_eq"], figsize=[20, 5])

### Finding the Lunar Minor cycle
Looking at the above plot, you can see there are peaks and troughs where the distance between the new moon and the equinox ranges from 0 to 30. Lets see if we can pull out these troughs in this chart by marking in the dataset when these troughs occur.

In [ ]:
# Filter all rows where the lunar_month = 1 and closest to the vernal equinox. 
# (this will be used in defining the lunar cycle)
mask = nm_df["lunar_month"] == 1
df = pd.DataFrame(nm_df[mask])
ilocs_min = argrelextrema(df["nm_eq"].values, np.less_equal)[0]  # [::9]

# set the minor lunar cycle based on the above filter
df.loc[df.iloc[ilocs_min].index, "minor_lunar_cycle"] = True
df[df.minor_lunar_cycle == True][:5]

Looks like when considering only the dates where the new moon and equinox are the closest. A lunar minor cycle is discovered.
Just like the lunar month can be 29 or 30 days or the lunar year 12 or 13 months. This minor cycle can be either 2 or 3 lunar years.

To better visualize this, we're going to add a running day incrementor between all the points between each of the minor lunar cycles

In [ ]:
mask = df["minor_lunar_cycle"] == True
nm2 = nm_df.merge(df[mask][["lunar_month", "lunar_year", "minor_lunar_cycle"]], on=["lunar_month", "lunar_year"], how="outer")
nm2['minor_lunar_cycle'] = nm2['minor_lunar_cycle'].fillna(False)

nm2['group'] = nm2['minor_lunar_cycle'].cumsum()
nm2['lunar_minor_cycle_year'] = nm2.groupby('group').cumcount() + 1
nm2.loc[nm2['minor_lunar_cycle'], 'lunar_minor_cycle_year'] = 1
nm2 = nm2.drop(columns=['group'])

nm2[:800].plot("lunar_year", y=["lunar_minor_cycle_year"], figsize=[20, 5])

### Patterns Emerge
Lets see what happens when we overlay the above chart with the 294 year cycle. We're going to zoom
in so we can see if there are any alignments

In [ ]:
df_filter = nm2[3200:4000]

fig, ax = plt.subplots(figsize=(20, 5))
ax2 = ax.twinx()

ax.plot(df_filter["sun_year"], df_filter["lunar_minor_cycle_year"])
ax2.plot(df_filter["sun_year"], df_filter["eyear"], color="orange")

### Lunar Greater Cycle
Amazingly enough it looks like this minor lunar cycle perfectly aligns with at least this first instance of the 294 year cycle.
Notice how when the 294 year cycle reset, it reset exactly on one of the troughs of this lunar cycle.

Lets take this a bit further, there is another lunar cycle hidden in here which I call the greater lunar cycle. This cycle is a cycle of either 32 or 33 lunar years

In [ ]:
mask = df["minor_lunar_cycle"] == True
nm2 = nm_df.merge(df[mask][["lunar_month", "lunar_year", "minor_lunar_cycle"]], on=["lunar_month", "lunar_year"], how="outer")
nm2['minor_lunar_cycle'] = nm2['minor_lunar_cycle'].fillna(False)
phoenix_cycle = nm2[nm2["minor_lunar_cycle"] == True][::12].index
nm2.loc[nm2.iloc[phoenix_cycle].index, "new_phoenix_cycle"] = True
nm2['new_phoenix_cycle'] = nm2['new_phoenix_cycle'].fillna(False)

nm2['group'] = nm2['new_phoenix_cycle'].cumsum()
nm2['lunar_cycle_year'] = nm2.groupby('group').cumcount() + 1
nm2.loc[nm2['new_phoenix_cycle'], 'lunar_cycle_year'] = 1

nm2 = nm2.drop(columns=['group', 'new_phoenix_cycle', 'minor_lunar_cycle'])
nm_df = nm2

In [ ]:
df_filter = nm_df[nm_df.lunar_month == 1][0:1000]

fig, ax = plt.subplots(figsize=(20, 5))
ax2 = ax.twinx()

ax.plot(df_filter["sun_year"], df_filter["lunar_cycle_year"])
ax2.plot(df_filter["sun_year"], df_filter["eyear"], color="orange")

In [ ]:
nm_df[(nm_df.lunar_month == 1) & (nm_df.eyear == 1)]

# Close but not the cigar
So far it looks like we got it. An amazing coincidence or did our ancestors know a little something about astronomy.
But let dig in to this a bit more. It's hard to see if the alignment is perfect because in the above chart we're looking at around 1000 years. Lets zoom in to right where the alignment occurs and see if it's perfect or not.


In [ ]:
cycle = 43
mult = 294
st = 200 + mult * cycle
df_filter = nm_df[nm_df.lunar_month == 1][st:st+100]

# df_filter = nm_df[nm_df.lunar_month == 1][270:320]
# df_filter = nm_df[nm_df.lunar_month == 1][570:620]
# df_filter = nm_df[nm_df.lunar_month == 1][870:920]


fig, ax = plt.subplots(figsize=(20, 5))
ax2 = ax.twinx()

ax.plot(df_filter["sun_year"], df_filter["lunar_cycle_year"])
ax2.plot(df_filter["sun_year"], df_filter["eyear"], color="red")

In [ ]:
nm_df[nm_df.lunar_month == 1][st:st+30]